# Extract Features for Train Set

In [ ]:
import keras
from keras.datasets import cifar10
import numpy as np
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input
import scipy
from scipy import misc
import skimage
from skimage import transform
import os

# load the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = np.squeeze(y_train) # get (50000,) instead of (50000,1)
print('data loaded')

# load MobileNetV2 model + remove final classification layers
model = MobileNetV2(weights='imagenet', include_top=False, 
                    input_shape=(224, 224,3))
print('model loaded')

In [ ]:
#model.summary()
print(x_train.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
path = ''

# obtain train features, read from file if already gotten
if os.path.exists(path + 'mnv2_features_train.npz'):
    print('features detected (train)')
    features = np.load(path + 'mnv2_features_train.npz')['features']
else:
    print('features file not detected (train)')
    print('calculating now ...')
    # first convert to correct size (upsize) and then to float32
    big_x_train = np.array([scipy.misc.imresize(x_train[i], (224, 224, 3)) 
                            for i in range(0, len(x_train))]).astype('float32') 
    # pre-processing is specific to each network
    mnv2_input_train = preprocess_input(big_x_train)
    print('train data preprocessed')
    # predict and save features
    features = model.predict(mnv2_input_train, verbose=1)
    features = np.squeeze(features)
    # save features in an uncompressed archive 
    np.savez(path + 'mnv2_features_train', features=features)
    print('train features saved (train)')

print('features ready (train)')
print(features.shape)

# Extract Bottleneck Features for Test Set

In [ ]:
# obtain features (test)
if os.path.exists(path + 'mnv2_features_test.npz'):
    print('features detected (test)')
    features_test = np.load('mnv2_features_test.npz')['features_test']
else:
    print('features file not detected (test)')
    print('calculating now ...')
    # first convert to correct size (upsize) and then to float32
    big_x_test = np.array([scipy.misc.imresize(x_test[i], (224, 224, 3)) 
                       for i in range(0, len(x_test))]).astype('float32')
    # pre-processing is specific to each network
    mnv2_input_test = preprocess_input(big_x_test)
    # predict and save features (test)
    features_test = model.predict(mnv2_input_test, verbose=1)
    features_test = np.squeeze(features_test)
    np.savez(path + 'mnv2_features_test', features_test=features_test)
print('features saved (test)')

# Train a Shallow NN

In [ ]:
from keras.utils import np_utils

# one-hot encode the labels
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [ ]:
print(y_train.shape)

In [ ]:
from keras.callbacks import ModelCheckpoint   
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, Flatten

D = 7*7*1280
#X_train = features.reshape(50000, D)
#print(X_train.shape)

mlp = Sequential()
mlp.add(Conv2D(filters=128, kernel_size=1, strides=1, 
               input_shape=(7, 7, 1280)))
mlp.add(Flatten())
mlp.add(Dense(100, activation='relu', input_shape=(D,)))
mlp.add(Dense(10, activation='softmax'))

mlp.summary()

mlp.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath= path + 'model.best.hdf5', 
                               verbose=1, save_best_only=True)

log = mlp.fit(features, y_train, batch_size=128, epochs=10,
          callbacks=[checkpointer], verbose=1, validation_data=(features_test, y_test))

In [ ]:
# load the weights that yielded the best validation accuracy
mlp.load_weights(path + 'model.best.hdf5')

# evaluate test accuracy
X_test = features_test.reshape(10000, D)
score = mlp.evaluate(features_test, y_test, verbose=1)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy: %.4f%%' % accuracy)

In [ ]:
import matplotlib.pyplot as pl
pl.plot(log.history['acc'], label='Training')
pl.plot(log.history['val_acc'], label='Testing')
pl.legend()
pl.grid()